# 4.1 인스타그램 크롤링 

### 4.1.2 인스타그램 검색 결과 URL을 만들어서 접속하기

In [ ]:
# 예제 4-1 인스타그램 검색결과 URL을 만드는 함수
def insta_searching(word):
    url = "https://www.instagram.com/explore/tags/" + word
    return url
    

In [ ]:
# 예제 4-2 selenium으로 URL 접속하기
from selenium import webdriver

driver = webdriver.Chrome("c:/playwithdata/chromedriver.exe")   

word = "제주도맛집"
url = insta_searching(word)
driver.get(url)


### 4.1.3 첫 번째 게시글 열기 

In [ ]:
# 예제 4-3 HTML에서 첫번째 게시글 찾아 클릭하기
def select_first(driver):
    first = driver.find_element_by_css_selector("div._9AhH0")
    first.click()
    time.sleep(3)

### 4.1.4 게시글 정보 가져오기 

In [ ]:
# 예제 4-4 게시글 정보 가져오기
import re

def get_content(driver):
    # ① 현재 페이지 html 정보 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    # ② 본문 내용 가져오기
    try:
        content = soup.select('div.C4VMK > span')[0].text
    except:
        content = ' '
    # ③ 본문 내용에서 해시태그 가져오기(정규식 활용)
    tags = re.findall(r'#[^\s#,\\]+', content)  
    # ④ 작성일자 정보 가져오기
    date = soup.select('time._1o9PC.Nzb55')[0]['datetime'][:10]
    # ⑤ 좋아요 수 가져오기
    try:
        like = soup.select('div.Nm9Fw > button')[0].text[4:-1]   
    except:
        like = 0
    # ⑥ 위치정보 가져오기
    try: 
        place = soup.select('div.M30cS')[0].text
    except:
        place = ''
    # ⑦ 수집한 정보 저장하기
    data = [content, date, like, place, tags]
    return data


### 4.1.5 다음 게시글 열기

In [ ]:
# 예제 4-5 다음 게시글 열기
def move_next(driver):

    right = driver.find_element_by_css_selector ('a.HBoOv.coreSpriteRightPaginationArrow')
    right.click()
    time.sleep(3)


### 4.1.6 여러 게시글 정보 수집하기

In [ ]:
# 예제 4-6 인스타그램 크롤링
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re

# ① 크롬 브라우저 열기
driver = webdriver.Chrome('c:/playwithdata/chromedriver.exe') 

# ② 인스타그램 검색페이지 URL 만들기
word = "제주도맛집"    #검색어
url = insta_searching(word)

# ③ 검색페이지 접속하기
driver.get(url)
time.sleep(3)

# ④ 첫 번째 게시글 열기
select_first(driver)

# ⑤ 비어있는 변수(results)만들기
results = [ ]


# ⑥→⑦→⑧ 여러 게시물 수집하기
target = 50      # 크롤링할 게시글 수
for i in range(target):
    data = get_content(driver)    # 게시글 정보 가져오기
    results.append(data)
    move_next(driver)

print(results[:2])

In [ ]:
# 예제 4-7 크롤링 결과 저장하기

import pandas as pd

results_df = pd.DataFrame(results)
results_df.columns = ['content','data','like','place','tags']
results_df.to_excel('./files/3_1_crawling_jejudoMatJip.xlsx')


### 4.1.8 여러 엑셀 파일의 중복을 제거한 후 통합 저장


In [ ]:
# 예제 4-8 여러 개의 저장파일 통합하기
jeju_insta_df = pd.DataFrame( [ ] )

folder = './files/'
f_list = ['3_1_crawling_jejudoMatJip.xlsx', '3_1_crawling_jejudoGwanGwang.xlsx', '3_1_crawling_jejuMatJip.xlsx', '3_1_crawling_jejuYeoHang.xlsx']
for fname in f_list:
    fpath = folder + fname
    temp = pd.read_excel(fpath)
    jeju_insta_df.append(temp)

jeju_insta_df.columns =['content','data','like','place','tags']


In [ ]:
# 예제 4-9 중복 데이터 제거하고 저장하기
jeju_insta_df.drop_duplicates(subset = [ "content"] , inplace = True)
jeju_insta_df.to_excel('./files/3_1_crawling_raw.xlsx', index = False)